Run all  cells bellow.

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os 
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [ ]:
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)


In [ ]:
actions = list(range(11))
actions = [str(action) for action in actions]
actions

In [ ]:
labels_dict = {num:label for num, label in enumerate(actions)}

In [ ]:
model_dict = pickle.load(open('./model_3.p', 'rb'))
model = model_dict['model_3']

cap = cv2.VideoCapture(0)
counter = 0
while True:

    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_rgb = cv2.flip(frame_rgb,1)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame_rgb,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            data_aux = np.array([[points.x, points.y] for points in hand_landmarks.landmark]).flatten()


        prediction = model.predict([np.asarray(data_aux)])

        predicted_character = labels_dict[int(prediction[0])]
        cv2.putText(frame_rgb, predicted_character, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                            cv2.LINE_AA)
        
#     cv2.imwrite(f'{counter}.jpg', cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR))
    cv2.imshow('frame', cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR))
    counter +=1
    if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()